In [1]:
import requests
#This function performs a search on the FDIC's BankFind API
#Inputs:
#    'search_type' - this is a keyword for one of six API endpoints
#        type - string
#        must be one of:
#            'institutions' - get financial institutions
#            'locations' - get institution locations
#            'history' - get detail on structure change events
#            'financials' - get quarterly financial information for FDIC insured institutions
#            'summary' - get historical aggregate data by year
#            'failures' - get detail on historical bank failures
#    'filters' - this is the query string syntax filter used to filter searchs
#        type - string
#        example - 'STNAME: "North Carolina"'
#    'fields' - this specifies which fields you want to be returned in your search
#        type - list of strings
#        example = ['CERT', 'ASSET']
#    'other_args' - any other arguments provided by the API that you'd like to provide in your search
#        type - dictionary
#        example - {'sort_by' : 'NAME'}
#Output:
#    Output is a list of search results retrieved from Bankfind API, according to filters and parameters
#        type - list of dictionaries
#        example - search('institutions', 'CERT: 139', ['CERT', 'STNAME']) returns 
#            [{'CERT' : 139, 'STNAME' : 'FLORIDA', 'ID' : '139'}]
#        note - 'ID' field is always returned regardless of 'fields' specification
#Note on search limit:
#    As recommended by the API docs, this function does not fetch more than 500 results per API call...
#    if more than 500 results match a search, the result is fetched in batches of 500, aggregated, and returned alltogether...
#    so, the user need not worry about the return limit, though searches of large size will take very long..
#    and should be avoided. For example, searching for all institutions will return over 27,000 results...
#    which can take the function as long as a minute to finish.
#    Avoid most searches larger than this (such as searching all financial data, over 1.5M results!!!)...
#    as they will take unreasonable amounts of time to complete
#For a full list of input/output fields and possible arguments, see BankFind API Docs at https://banks.data.fdic.gov/docs/

def search(search_type, filters='', fields = [], other_args = {}):
    base_URL = 'https://banks.data.fdic.gov/api/'
    finished = False
    data = []
    
    params = {'limit' : 500}
    params['filters'] = filters
    params['offset'] = 0
    params['fields'] = ','.join(fields)
    
    for key, value in other_args.items():
        params[key] = value
    
    while not finished:
        try:
            r = requests.get(base_URL + search_type, params=params).json()
        except requests.exceptions.ConnectionError:
            print("Connection error")
            continue
        data.extend(r['data'])
        if len(r['data']) < params['limit']:
            finished = True
        params['offset'] += params['limit']
        
    return [x['data'] for x in data]